## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import datetime as dt

import time

import yfinance as yf

import os

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

from plotly.subplots import make_subplots

from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")

## Constants

In [3]:
PATH = "Data/"

# Start End date defaults
S_DATE = "2012-01-01"
E_DATE = "2023-01-01"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)

## Get Dataframe from CSV

In [4]:
# Reads a dataframe from CSV file, changes index to date and returns it
def get_stock_df_from_csv(ticker):

    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col=0)
    except FileNotFoundError:
        print("File doesn't exist")
    else:
        return df

## Get all Stocks Downloaded in List

In [5]:
# Get all files and extract tickers from them
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers.sort()
len(tickers)

4105

## Add Daily Returns

In [6]:
# Shift provides the value from the previous day
# NaN is displayed because there was no previous day price for the 1st calculation
def add_daily_return_to_df(df):
    df['daily_return'] = (df['Close']/df['Close'].shift(1)) - 1

    # Save data to a CSV file
    # df.to_csv(PATH + ticker + '.csv')
    return df

## Add Cumulative Returns

In [7]:
def add_cum_return_to_df(df):
    df['cum_return'] = (1 + df['daily_return']).cumprod()
    # df.to_csv(PATH + ticker + '.csv')
    return df

## Add Bollinger Bands

An indicator of volatility.

Bollinger Bands plot 2 lines using a moving average and the standard deviation defines how far apart the lines are. They also are used to define if prices are to high or low. When bands tighten it is believed a sharp price move in some direction. Prices tend to bounce off of the bands which provides potential market actions.

A strong trend should be noted if the price moves outside the band. If prices go over the resistance line it is in overbought territory and if it breaks through support it is a sign of an oversold position.

In [8]:
# Here we will add a middle band (20 days), upper band (20 days + 1.96*std),
# and lower band (20 days - 1.96*std)
def add_bollinger_bands(df):
    df['middle_band'] = df['Close'].rolling(window=20).mean()
    df['upper_band'] = df['middle_band'] + 1.96*df['Close'].rolling(window=20).std()
    df['lower_band'] = df['middle_band'] - 1.96*df['Close'].rolling(window=20).std()
    # df.to_csv(PATH + ticker + '.csv')
    return df

## Add Ichimoku Data to Dataframe

The Ichimoku (One Look) is considered an all in one indicator. It provides information on momentum, support and resistance. It is made up of 5 lines. If you are a short term trader you create 1 minute or 6 hour. Long term traders focus on day or weekly data.

- Conversion Line (Tenkan-sen) : Represents support, resistance and reversals. Used to measure short term trends.
- Baseline (Kijun-sen) : Represents support, resistance and confirms trend changes. Allows you to evaluate the strength of medium term trends. Called the baseline because it lags the price.
- Leading Span A (Senkou A) : Used to identify future areas of support and resistance
- Leading Span B (Senkou B) : Other line used to identify future support and resistance
- Lagging Span (Chikou) : Shows possible support and resistance. It is used to confirm signals obtained from other lines.
- Cloud (Kumo) : Space between Span A and B. Represents the divergence in price evolution.

Formulas

- Lagging Span = Price shifted back 26 periods
- Base Line = (Highest Value in period + Lowest value in period)/2 (26 Sessions)
- Conversion Line = (Highest Value in period + Lowest value in period)/2 (9 Sessions)
- Leading Span A = (Conversion Value + Base Value)/2 (26 Sessions)
- Leading Span B = (Conversion Value + Base Value)/2 (52 Sessions)

In [9]:
def add_Ichimoku(df):
    # Conversion
    hi_val = df['High'].rolling(window=9).max()
    low_val = df['Low'].rolling(window=9).min()
    df['Conversion'] = (hi_val + low_val)/2

    # Baseline
    hi_val2 = df['High'].rolling(window=26).max()
    low_val2 = df['Low'].rolling(window=26).min()
    df['Baseline'] = (hi_val2 + low_val2)/2

    # Spans
    df['SpanA'] = ((df['Conversion'] + df['Baseline'])/2).shift(26)
    hi_val3 = df['High'].rolling(window=52).max()
    low_val3 = df['Low'].rolling(window=52).min()
    df['SpanB'] = ((hi_val3 + low_val3)/2).shift(26)
    df['Lagging'] = df['Close'].shift(-26)

    return df

## Add Daily, Cumulative Bollinger Bands & Ichimoku to Dataframes

In [11]:
for x in tickers:
    try:
        print("Working on: ", x)
        new_df = get_stock_df_from_csv(x)
        new_df = add_daily_return_to_df(new_df)
        new_df = add_cum_return_to_df(new_df)
        new_df = add_bollinger_bands(new_df)
        new_df = add_Ichimoku(new_df)
        new_df.to_csv(PATH + x + '.csv')
    except Exception as ex:
        print(ex)

Working on:  08ABB_BO
Working on:  08ADD_BO
Working on:  08ADR_BO
Working on:  08AGG_BO
Working on:  08AMD_BO
Working on:  08AMR_BO
Working on:  08AQD_BO
Working on:  08AQR_BO
Working on:  08BPB_BO
Working on:  08DPD_BO
Working on:  08DPR_BO
Working on:  08GPG_BO
Working on:  08MPD_BO
Working on:  08MPR_BO
Working on:  08QPD_BO
Working on:  08QPR_BO
Working on:  11ADD_BO
Working on:  11ADR_BO
Working on:  11AGG_BO
Working on:  11AMD_BO
Working on:  11AMR_BO
Working on:  11AQD_BO
Working on:  11AQR_BO
Working on:  11DPD_BO
Working on:  11DPR_BO
Working on:  11GPG_BO
Working on:  11MPD_BO
Working on:  11MPR_BO
Working on:  11QPD_BO
Working on:  11QPR_BO
Working on:  1STCUS_BO
Working on:  20MICRONS_BO
Working on:  21STCENMGM_BO
Working on:  3MINDIA_BO
Working on:  3PLAND_BO
Working on:  4THGEN_BO
Working on:  5PAISA_BO
Working on:  63MOONS_BO
Working on:  7NR_BO
Working on:  7SEASL_BO
Working on:  7TEC_BO
Working on:  A2ZINFRA_BO
Working on:  AADIIND_BO
Working on:  AAIL_BO
Working on:  